<center><strong>Sistem Temu Kembali Informasi</strong><br />
<strong><font color="blue">Semester Gasal T.A. 2020/2021</font></strong><br />
</center>

<strong>Outline pertemuan minggu ke-5</strong><br />
<li> Pendahuluan Klasifikasi </li>
<li> K-NN, Naive Byaes, Decision Tree, SVM, NN </li>
<li> Visualisasi dengan boxplot </li>


<p><img alt="" src="figures/56_Supervised VS unsupervised.png" style="height:500px; width:1153px" /></p>

### K-Nearest Neighbour (K-NN) 
- Classifier yang paling sederhana
- Dapat digunakan untuk permaslaahn klasifikasi dan regresi

<strong> Kelebihan: </strong>
- Efisien untuk data berukuran besar
- Sederhana, mudah diimplementasikan
- Mudah dimodikasi (perhitungan jarak)
- Menangani data multiclass dengan mudah

<strong> Kekurangan: </strong>
- Semakin besar data semakin lambat performanya (menghitung NN)
- Memerlukan jumlah K optimal
- Tidak cocok untuk imbalance data

<p><img alt="" src="figures/6_kNN.JPG" style="height:300px; width:711px" /></p>

In [ ]:
# load data pickle dari 20newsgroups yang didapat dari pertemuan 4
import pickle
import os

src_name = "20newsgroup.pckl"
src_path = os.path.join("data", src_name)
with open(src_path, 'rb') as fin:
    data = pickle.load(fin)

docs = [doc for doc in data.data]
label = data.target

print('DONE!')

In [ ]:
# preprocessing
import re
from nltk import sent_tokenize
from nltk import word_tokenize

def preprocess(doc):
    sents = sent_tokenize(doc)
    sents_tok = list() # tokenisasi kalimat
    for s in sents:
        s = s.strip().lower() # case folding dan menghilangkan new line
        s = s.replace("\n", " ") # menggantikan \n dengan spasi
        s = re.sub(r'[^a-zA-Z0-9 ]', ' ', s) # menghapus simbol
        s = re.sub(' +', ' ', s) # menghapus repetitive space
        
        sents_tok.append(s)
    
    return " ".join(sents_tok)

docs_clear = list()
for d in docs:
    docs_clear.append(preprocess(d))

In [ ]:
# representasi vektor dengan VSM-TFIDF
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2)
X = tfidf_vectorizer.fit_transform(docs_clear)
print(X.shape)

In [ ]:
print(len(label))
print(len(docs_clear))
print(len(X.getnnz(1)))
print(X.shape)

In [ ]:
# Cleaning noise
# Hapus label yang memiliki row = 0 pada tfidf-nya
# Hapus row yang memiliki nilai 0
Y = label[X.getnnz(1)>0]
X_clear = X[X.getnnz(1)>0]

# Hapus dokumen yang memiliki nilai row 0
docs_nonzero = list()
for i, d in enumerate(X.getnnz(1)>0):
    if d:
        docs_nonzero.append(docs_clear[i])

print(X_clear.shape, len(Y), len(docs_nonzero))

In [ ]:
# Membagi data menjadi dua yaitu data training dan data testing
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X_clear, Y, test_size=.3, random_state=7)
print(X_train.shape, X_test.shape)

In [ ]:
# KNN menggunakan scikit-learn library
# http://scikit-learn.org/stable/modules/neighbors.html
from sklearn import neighbors

K = 10
weights = 'distance'
kNN = neighbors.KNeighborsClassifier(n_neighbors, weights=weights)
kNN.fit(X_train, Y_train)
print('Done!')

In [ ]:
# Prediksi dengan KNN
Y_knn = kNN.predict(X_test)

In [ ]:
# Evaluasi dengan akurasi
from sklearn.metrics import accuracy_score

accuracy_score(Y_test, Y_knn)

<h3 id="Latihan:"><font color="blue">Latihan 1:</font></h3>

<ul> 
    <li> Mengapa perlu dilakukan data splitting (training dan testing)?</li>
    <li> Apakah rasio splitting berpengaruh pada performa dari model? </li>
    <li> Cobalah beberapa model dengan merubah nilai dari parameter K, berapakah nilai K yang paling optimal? </li>

In [ ]:
# Kerjakan latihan 1 pada cell berikut ini



### Naive Bayes Classifier

Metode klasifikasi dengan pendekatan probabilitas yang didasarkan pada teorema <strong>Bayes</strong>. Berasumsi bahwa semua fitur adalah independen dan memiliki kontribusi yang sama.

<p><img alt="" src="figures/naive_bayes.png" style="width: 400px ; height: 220px" /></p>

<strong>Dimana:</strong>
- x : fitur
- c : kelas
- P(c|x) : Probabilitas kelas c diberikan x sebagai fitur

<strong> Kelebihan:</strong>
- Cepat dan mudah diimplementasikan
- Cocok untuk multiclass
- Dapat bekerja dengan baik untuk data berdimensi tinggi (high-dimensional data)

<strong> Kekurangan:</strong>
- Asumsi bahwa semua fitur independen terkadang tidak sesuai dengan real-life data

In [ ]:
# Menggunakan data sebelumnya
# Naive bayes menggunakan library scikit-learn
# http://scikit-learn.org/stable/modules/naive_bayes.html
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()
nbc = gnb.fit(X_train.toarray(), Y_train) 
Y_nbc = nbc.predict(X_test.toarray())

accuracy_score(Y_test, Y_nbc)

### Decision Tree

Decision tree menggunakan rule-based (if else condition) pada sebuah tree untuk membangan sebuah model. Decision tree dapat dipergunakan untuk categorical ataupun numerical data. 

<p><img alt="" src="figures/6_DT.png" style="height:336px; width:904px" /></p>

<p> Contoh: </p>
<p><img alt="" src="figures/6_DT_meme.png" style="height:342px; width:460px" /></p>

<strong>Kelebihan:</strong>
- Mudah untuk dipahami, cocok untuk visualisasi
- Dapat digunakan untuk categorical maupun numerical data
- non-parametric model
- Robust terhadap outliers dan missing values

<strong>Kekurangan:</strong>
- Sering overfit





In [ ]:
# Decision Tree dengan library scikit-learn
#http://scikit-learn.org/stable/modules/tree.html
from sklearn import tree

DT = tree.DecisionTreeClassifier()
DT = DT.fit(X_train, Y_train)
Y_DT = DT.predict(X_test)

accuracy_score(Y_test, Y_DT)

In [ ]:
!pip install pydotplus

In [ ]:
# Visualisasi tree
import pydotplus
from sklearn.externals.six import StringIO
from IPython.display import Image
from sklearn.tree import export_graphviz

dot_data = StringIO()
export_graphviz(DT, out_file=dot_data, filled=True, rounded=True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
Image(graph.create_png())

### Random Forest

<p><img alt="" src="figures/5_RandomForest.png" style="width: 592px; height: 444px;" /></p>

<p>Terdiri dari beberapa decision tree. Merupakan ensamble algorithm yang mengambil hasil dari beberapa decision tree. </p>

<strong>Kelenihan:</strong>
- Robust terhadap outlier
- Dapat bekerja dengan baik pada non-linear data
- Dapat menurunkan risiko overfitting
- Dapat bekerja dengan efisien pada data set yang besar

<strong>Kekurangan:</strong>
- Bias ketika dipergunakan untuk categorical variabel
- lambat

In [ ]:
# Random Forest
# http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
from sklearn.ensemble import RandomForestClassifier

RandomForest = RandomForestClassifier()
RandomForest.fit(X_train, Y_train)
Y_RF = RandomForest.predict(X_test)
accuracy_score(Y_test, Y_RF)

### Support Vector Machine (SVM)

<p><img alt="" src="figures/6_SVM.jpg" style="height: 262px ; width: 232px" /><img alt="" src="figures/svm_opt.png" style="width: 300px; height: 106px;" /></p>
<p>SVM melakukan plotting data ke dalam n-dimensi. Klasifikasi dilakukan dengan mencari hyper-plane yg memisahkan dua kelas.</p>

<h4> SVM kernel </h4>
<p> SVM kernel digunakan untuk data yang bersifat non-linear. Data yang bersifat non-linear tidak dapat dipisahkan dengan menggunakan garis lurus seperti pada data linear. Untuk mengatasi permasalahan tersebut digunakan <strong>kernel trick</strong>. Kernel trick mentransformasikan data ke dalam dimensi yg lebih tinggi agar lebih mudah dipisahkan.</p>

<strong> Beberapa kernel SVM:</strong>
- Linear
- Polynomial
- RBF
- Sigmoid


<h4>SVM multiclass</h4>

<p><img alt="" src="figures/6_SVM_Ova.jpg" style="height:314px; width:432px" /></p>

<strong>Kelebihan:</strong>
- Dapat bekerja dengan efektif pada dataset berukuran kecil
- Memiliki performa yang baik untuk data berdimensi tinggi
- Relatif efisien dalam penggunaan memori

<strong>Kekurangan:</strong>
- Kurang cocok untuk dataset berukuran besar
- Kurang efektif untuk data yang memiliki banyak noise

In [ ]:
# SVM
# http://scikit-learn.org/stable/modules/svm.html
from sklearn import svm

dSVM = svm.SVC(decision_function_shape='ovo') # one versus one SVM
dSVM.fit(X_train, Y_train)
Y_SVM = dSVM.predict(X_test)
accuracy_score(Y_test, Y_SVM)

In [ ]:
# Menggunakan kernel trick
# Default kernel dari fungsi SVM pada scikit learn adalah 'RBF'
list_kernel = ['linear', 'poly', 'rbf', 'sigmoid']
for kernel in list_kernel:
    dSVM = svm.SVC(kernel=kernel, gamma='auto')
    dSVM.fit(X_train, Y_train)
    y_SVM = dSVM.predict(X_test)
    print(kernel, ':', accuracy_score(Y_test, y_SVM))

### Cross Validation

<p>Cross validation dipergunakan untuk memvalidasi hasil dari suatu model, memastikan objektifitas dari hasil. </p>

<p><img alt="" src="figures/6_Cross_validation.png" style="height:274px; width:485px" /></p>

In [ ]:
# Cross validation
# http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html
from sklearn.model_selection import cross_val_score
import time

svm_ = svm.SVC(kernel='linear')
start = time.time()
scores_svm = cross_val_score(svm_, X_clear, Y, cv=10) # perhatikan sekarang kita menggunakan seluruh data
timepros = time.time() - start

print("Accuracy SVM: %0.2f (+/- %0.2f), Waktu = %0.3f detik" % (scores_svm.mean(), scores_svm.std() * 2, timepros))

In [ ]:
# Bagaimana dengan k-NN ?
mulai = time.time()
scores_kNN = cross_val_score(kNN, X_clear, Y, cv=10) # perhatikan sekarang kita menggunakan seluruh data
waktu = time.time() - mulai
# Interval Akurasi 95 CI 
print("Accuracy k-NN: %0.2f (+/- %0.2f), Waktu = %0.3f detik" % (scores_kNN.mean(), scores_kNN.std() * 2, waktu))

In [ ]:
# Visualisasi hasil dengan boxplot
# Buat BoxPlotnya
import matplotlib.pyplot as plt
import numpy as np

Data = [scores_svm, scores_kNN]
L = ['SVM', 'kNN']

fig, ax = plt.subplots()
  
# Creating plot 
plt.boxplot(Data) 
  
# show plot 
plt.show() 

### Bagaimana jika ingin menggunakan word embedding sebagai representasi vektornya?

In [ ]:
# Melakukan persiapan dokumen
# yang digunakan sebagai data adalah docs_nonzero
doc_gensim = [word_tokenize(d) for d in docs_nonzero]

In [ ]:
# membuat Word Embeddingnya
from gensim.models import FastText

N = len(doc_gensim)
dim = 300 # Jumlah neurons = ukuran vektor = jumlah kolom
model_FT = FastText(doc_gensim, size=dim, window=5, min_count=1, workers=-1)

In [ ]:
# NOTE: 
# tidak dapat langsung menggunakan hasil dari word embedding
# representasi vektor embedding adalah representasi per kata
# harus mengkonversikan dari word embedding ke dalam vektor space
# dokumen x words
def we2vsm(model_we, data_we):
    N = len(data_we)
    L = model_we.vector_size
    vsm_we = np.empty([N, L], dtype=np.float64) # inisialisasi matriks
    for i,d in enumerate(data_we):
        tmp = np.zeros([1, L], dtype=np.float64)
        count = 0
        for t in d:
            try:
                tmp += model_we.wv.__getitem__([t])
                count += 1
            except:
                pass
        if count>0:
            vsm_we[i] = tmp/count
    return vsm_we

In [ ]:
vsm_we = we2vsm(model_FT, doc_gensim)
vsm_we.shape

#### Setelah memiliki bentuk representasi dokumen, langkah selanjutnya adalah melakukan proses klasifikasi seperti yang telah dicontohkan sebelumnya

<h3 id="Latihan:"><font color="blue">Latihan 2:</font></h3>

- Lakukan klasifikasi dengan representasi yang telah diperoleh dari proses word embedding (vsm_we) dengan berbagai metode (knn, desicion tree, random forest, naive bayes, dan VSM)
- Visualisasikan hasilnya
- Bandingkan hasilnya dengan representasi dari VSM-TFIDF

In [ ]:
# Kerjakan latihan 2 pada cell berikut ini

